## Biblioteki

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from collections import deque
from openpyxl import load_workbook

## Pobór danych 

In [ ]:
# file_path1 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_48.xlsx"
# file_path2 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_76.xlsx"
# file_path3 = "C:/Users/olgas/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_127.xlsx"

file_path1 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_48.xlsx"
file_path2 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_76.xlsx"
file_path3 = "C:/Users/Justyna/source/repos/Projekt_IO/TSP-problem/Dane_TSP_127.xlsx"

# file_path1 = "C:/Users/ameli/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_48.xlsx"
# file_path2 = "C:/Users/ameli/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_76.xlsx"
# file_path3 = "C:/Users/ameli/OneDrive/Documents/GitHub/TSP-problem/Dane_TSP_127.xlsx"

# file_path1 = "C:/Users/wera6/Downloads/Dane_TSP_48.xlsx"
# file_path2 = "C:/Users/wera6/Downloads/Dane_TSP_76.xlsx"
# file_path3 = "C:/Users/wera6/Downloads/Dane_TSP_127.xlsx"

# index_col=0 zamienia pierwszą kolumne na indeksy wierszy 
# .to_numpy() zamienia ramkę danych na macierz
data1 = pd.read_excel(file_path1, index_col=0).to_numpy()

data2 = pd.read_excel(file_path2, index_col=0).to_numpy()

data3 = pd.read_excel(file_path3, index_col=0).to_numpy()

## Długość ścieżki

In [ ]:
def calculate_path_cost(matrix, path):
    return sum(matrix[path[i - 1]][path[i]] for i in range(len(path))) + matrix[path[-1]][path[0]]

## Algorytm genetyczny (GA)

In [ ]:
def crossover_order(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None] * size
    child[start:end] = parent1[start:end]

    pointer = 0
    for gene in parent2:
        if gene not in child:
            while child[pointer] is not None:
                pointer += 1
            child[pointer] = gene

    return child

def crossover_pmx(parent1, parent2):
    size = len(parent1)
    start, end = 2, 6
    child = [None] * size

    # Copy segment from parent1 to child
    child[start:end] = parent1[start:end]

    # Fill remaining positions from parent2
    for i in range(size):
        if child[i] is None:
            gene = parent2[i]
            while gene in child:
                gene = parent2[parent1.index(gene)]
            child[i] = gene

    return child

def selection_tournament(population, scores, k=3):
    selected = random.sample(range(len(population)), k)
    selected = sorted(selected, key=lambda x: scores[x])
    return population[selected[0]]

def selection_roulette(population, scores):
    total_score = sum(scores)
    
    # Calculate probabilities
    probabilities = [score / total_score for score in scores]
    print(probabilities)
    # Calculate cumulative probabilities
    cumulative_probabilities = [sum(probabilities[:i+1]) for i in range(len(probabilities))]
    print(cumulative_probabilities)
    # Pick a random number between 0 and 1
    pick = random.uniform(0, 1)
    print(pick)
    # Select the individual based on the random number
    for i, cumulative_probability in enumerate(cumulative_probabilities):
        if pick <= cumulative_probability:
            return population[i]


def mutate_swap(individual):
    i, j = random.sample(range(len(individual)), 2)
    individual[i], individual[j] = individual[j], individual[i]

def mutate_inversion(individual):
    i, j = sorted(random.sample(range(len(individual)), 2))
    individual[i:j] = reversed(individual[i:j])

def mutate_insertion(individual):
    i, j = random.sample(range(len(individual)), 2)
    gene = individual.pop(i)
    individual.insert(j, gene)


def genetic_algorithm(distance_matrix, population_size=10, generations=5000, crossover_rate=0.6, mutation_rate=0.15, selection_method='tournament', crossover_method='order', mutation_method='swap'):
    start_time = time.time()

    n = len(distance_matrix)
    population = [random.sample(range(n), n) for _ in range(population_size)]
    best_solution = None
    best_cost = float('inf')
    best_time = 0

    for generation in range(1, generations + 1):
        scores = [calculate_path_cost(distance_matrix, individual) for individual in population]
        new_population = []

        for _ in range(population_size // 2):
            if selection_method == 'tournament':
                parent1 = selection_tournament(population, scores)
                parent2 = selection_tournament(population, scores)
            elif selection_method == 'roulette':
                parent1 = selection_roulette(population, scores)
                parent2 = selection_roulette(population, scores)

            if random.random() < crossover_rate:
                if crossover_method == 'order':
                    child1 = crossover_order(parent1, parent2)
                    child2 = crossover_order(parent2, parent1)
                elif crossover_method == 'pmx':
                    child1 = crossover_pmx(parent1, parent2)
                    child2 = crossover_pmx(parent2, parent1)
            else:
                child1, child2 = parent1[:], parent2[:]

            if random.random() < mutation_rate:
                if mutation_method == 'swap':
                    mutate_swap(child1)
                    mutate_swap(child2)
                elif mutation_method == 'inversion':
                    mutate_inversion(child1)
                    mutate_inversion(child2)
                elif mutation_method == 'insertion':
                    mutate_insertion(child1)
                    mutate_insertion(child2)

            new_population.extend([child1, child2])

        population = new_population

        for individual in population:
            cost = calculate_path_cost(distance_matrix, individual)
            if cost < best_cost:
                best_time = time.time() - start_time
                best_solution = individual
                best_cost = cost

        if (generation % 100 == 0) or (generation == generations):
            elapsed_time = time.time() - start_time  # Calculate elapsed time
            print(f"Iteration {generation}, Best Cost: {best_cost: .2f}, Elapsed Time: {elapsed_time:.2f} seconds")

    return best_solution, best_cost, best_time

### Generowanie wyników

Za podstawowe dane przyjmujemy:

* population_size = 10 

* generations = 5000

* crossover_rate = 0.6

* mutation_rate = 0.15

* selection_method = `tournament`
    
* crossover_method = `order`
    
* mutation_method = `swap`

In [ ]:
# Dane_TSP_48
best_path, best_cost, best_time = genetic_algorithm(
    data1,
    population_size=10,
    generations=5000,
    crossover_rate=0.6,
    mutation_rate=0.15,
    selection_method='tournament',
    crossover_method='order',
    mutation_method='swap'
)

In [ ]:
# Dane_TSP_76
best_path, best_cost, best_time = genetic_algorithm(
    data2,
    population_size=10,
    generations=5000,
    crossover_rate=0.6,
    mutation_rate=0.15,
    selection_method='tournament',
    crossover_method='order',
    mutation_method='swap'
)

In [ ]:
# Dane_TSP_127
best_path, best_cost, best_time = genetic_algorithm(
    data3,
    population_size=10,
    generations=5000,
    crossover_rate=0.6,
    mutation_rate=0.15,
    selection_method='tournament',
    crossover_method='order',
    mutation_method='swap'
)

#### Badanie wpływu poszczególnych parametrów

In [ ]:
def test_genetic_algorithm(data, sheet_prefix, writer):
    # Definicja różnych wartości parametrów
    population_sizes = [5, 10, 15, 20]
    mutation_rates = [0.01, 0.05, 0.1, 0.2]
    crossover_rates = [0.6, 0.7, 0.8, 0.9]
    generations_list = [5000, 7500, 10000, 15000]
    selection_methods = ['roulette', 'tournament']
    crossover_methods = ['order', 'pmx']
    mutation_methods = ['insertion', 'swap', 'inversion']

    # Testowanie wpływu population_size
    results = []
    for population_size in population_sizes:
        print(f"Testing with population_size={population_size}")
        best_path, best_cost, best_time = genetic_algorithm(data, population_size=population_size)
        results.append({
            'parameter': 'population_size',
            'value': population_size,
            'best_path': best_path,
            'best_cost': best_cost,
            'best_time': best_time
        })
    df = pd.DataFrame(results)
    df.to_excel(writer, sheet_name=f'{sheet_prefix}_population_size', index=False)

    # Testowanie wpływu mutation_rate
    results = []
    for mutation_rate in mutation_rates:
        print(f"Testing with mutation_rate={mutation_rate}")
        best_path, best_cost, best_time = genetic_algorithm(data, mutation_rate=mutation_rate)
        results.append({
            'parameter': 'mutation_rate',
            'value': mutation_rate,
            'best_path': best_path,
            'best_cost': best_cost,
            'best_time': best_time
        })
    df = pd.DataFrame(results)
    df.to_excel(writer, sheet_name=f'{sheet_prefix}_mutation_rate', index=False)

    # Testowanie wpływu crossover_rate
    results = []
    for crossover_rate in crossover_rates:
        print(f"Testing with crossover_rate={crossover_rate}")
        best_path, best_cost, best_time = genetic_algorithm(data, crossover_rate=crossover_rate)
        results.append({
            'parameter': 'crossover_rate',
            'value': crossover_rate,
            'best_path': best_path,
            'best_cost': best_cost,
            'best_time': best_time
        })
    df = pd.DataFrame(results)
    df.to_excel(writer, sheet_name=f'{sheet_prefix}_crossover_rate', index=False)

    # Testowanie wpływu generations
    results = []
    for generations in generations_list:
        print(f"Testing with generations={generations}")
        best_path, best_cost, best_time = genetic_algorithm(data, generations=generations)
        results.append({
            'parameter': 'generations',
            'value': generations,
            'best_path': best_path,
            'best_cost': best_cost,
            'best_time': best_time
        })
    df = pd.DataFrame(results)
    df.to_excel(writer, sheet_name=f'{sheet_prefix}_generations', index=False)

    # Testowanie wpływu selection_method
    results = []
    for selection_method in selection_methods:
        print(f"Testing with selection_method={selection_method}")
        best_path, best_cost, best_time = genetic_algorithm(data, selection_method=selection_method)
        results.append({
            'parameter': 'selection_method',
            'value': selection_method,
            'best_path': best_path,
            'best_cost': best_cost,
            'best_time': best_time
        })
    df = pd.DataFrame(results)
    df.to_excel(writer, sheet_name=f'{sheet_prefix}_selection_method', index=False)

    # Testowanie wpływu crossover_method
    results = []
    for crossover_method in crossover_methods:
        print(f"Testing with crossover_method={crossover_method}")
        best_path, best_cost, best_time = genetic_algorithm(data, crossover_method=crossover_method)
        results.append({
            'parameter': 'crossover_method',
            'value': crossover_method,
            'best_path': best_path,
            'best_cost': best_cost,
            'best_time': best_time
        })
    df = pd.DataFrame(results)
    df.to_excel(writer, sheet_name=f'{sheet_prefix}_crossover_method', index=False)

    # Testowanie wpływu mutation_method
    results = []
    for mutation_method in mutation_methods:
        print(f"Testing with mutation_method={mutation_method}")
        best_path, best_cost, best_time = genetic_algorithm(data, mutation_method=mutation_method)
        results.append({
            'parameter': 'mutation_method',
            'value': mutation_method,
            'best_path': best_path,
            'best_cost': best_cost,
            'best_time': best_time
        })
    df = pd.DataFrame(results)
    df.to_excel(writer, sheet_name=f'{sheet_prefix}_mutation_method', index=False)



# Otwieranie istniejącego pliku Excel lub tworzenie nowego
filename = 'GA_127.xlsx'
try:
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
except FileNotFoundError:
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')

# Testowanie dla każdego zestawu danych
# test_genetic_algorithm(data1, 'TSP_48', writer)
# test_genetic_algorithm(data2, 'TSP_76', writer)
test_genetic_algorithm(data3, 'TSP_127', writer)

# Zapisanie pliku Excel
writer.close()